In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from utils.helper_function import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

import optuna
import pickle
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=UserWarning)


/tmp/ipykernel_49833/240062032.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_sk = pd.read_csv("../data/eye-irritation/train.csv")
test_sk = pd.read_csv("../data/eye-irritation/val.csv")

In [3]:
train_sk.head()

,SMILES,Activity
0,CCOP(OC1=C(C=C(C=C1C)C(C)(C)C)C(C)(C)C)OC1=C(C...,0
1,CCCCCCCCCCCCCCCCSC1NC2=CC(=CC=C2N=1)S(O)(=O)=O,1
2,O=C(CC(=O)CC1=CC(F)=C(F)C=C1F)N1CC2=NN=C(N2CC1...,0
3,CC1C=C(N)N(N=1)C1C=CC=CC=1,0
4,CC1CCCC(C)(C)C=1C(=O)C=CC,0


In [4]:
test_sk.head()

,SMILES,Activity
0,COC1=CC=C(CC2CC2)C=C1,1
1,CC(C)OP(=O)(OC(C)C)SCC1C=CC=CC=1,1
2,NNC(N)=O,1
3,OS(=O)(=O)C1C=CC=C2C=C(C=C(NC3C=CC=CC=3)C2=1)N...,1
4,CCCC1COC(CC1)C1C=CC(O)=CC=1,1


In [5]:
smilesTrain = train_sk["SMILES"].to_numpy()
activityTrain = train_sk["Activity"].to_numpy()

smilesTest = test_sk["SMILES"].to_numpy()
activityTest = test_sk["Activity"].to_numpy()

In [6]:
idx = valid_indices(smilesTrain)
idx2 = valid_indices(smilesTest)

smilesTrain = smilesTrain[idx]
smilesTest = smilesTest[idx2]

activityTrain = activityTrain[idx]
activityTest = activityTest[idx2]

[23:48:49] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[23:48:49] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 18 19 20 21 22 23
[23:48:49] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18


[23:48:49] Can't kekulize mol.  Unkekulized atoms: 5 6 7 9 10 11 12 13 14 15 16 17 18
[23:48:49] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 17 19 20 21 25 27 28
[23:48:49] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15


In [7]:
print(np.concatenate((smilesTrain, smilesTest)).shape)

(3871,)


In [8]:
mordred_class = mordredWrapper (np.concatenate((smilesTrain, smilesTest)))

mordred_train, activityTrain = mordred_class.get_fingerprints(smiles_array= smilesTrain, labels= activityTrain)

mordred_test, activityTest = mordred_class.get_fingerprints(smiles_array= smilesTest, labels= activityTest)

  1%|          | 28/3871 [00:02<03:52, 16.50it/s] 

/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|▎         | 131/3871 [00:12<09:07,  6.83it/s]

/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  4%|▍         | 164/3871 [00:15<08:13,  7.51it/s]

/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  8%|▊         | 296/3871 [00:27<06:57,  8.55it/s]

/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 3871/3871 [05:42<00:00, 11.31it/s]
/home/seg0fied/Toxicity_seminar/src/../utils/helper_function.py:213: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  truth_map = df_mordred.applymap(lambda x : not isinstance(x, mordred.error.MissingValueBase))
/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [9]:
mordred_train = np.array(mordred_train) 
mordred_test = np.array(mordred_test)


In [10]:
def svm_modl(trial, xtrain, ytrain):
    n = trial.suggest_int('degree', 2, 5)
    model_svm = SVC(degree= n)

    scores = cross_validate(model_svm, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [11]:
study2 = optuna.create_study(study_name='SVM_study_eye_irritation_mordred', storage= None)  
study2.optimize(lambda trial: svm_modl(trial, mordred_train, activityTrain), n_trials=20)  

[I 2024-02-05 00:09:36,477] A new study created in memory with name: SVM_study_eye_irritation_mordred
[I 2024-02-05 00:10:45,077] Trial 0 finished with value: 1.4729701996034248 and parameters: {'degree': 2}. Best is trial 0 with value: 1.4729701996034248.
[I 2024-02-05 00:11:55,040] Trial 1 finished with value: 1.4729701996034248 and parameters: {'degree': 3}. Best is trial 0 with value: 1.4729701996034248.
[I 2024-02-05 00:13:04,667] Trial 2 finished with value: 1.4729701996034248 and parameters: {'degree': 4}. Best is trial 0 with value: 1.4729701996034248.
[I 2024-02-05 00:14:10,820] Trial 3 finished with value: 1.4729701996034248 and parameters: {'degree': 2}. Best is trial 0 with value: 1.4729701996034248.
[I 2024-02-05 00:15:17,329] Trial 4 finished with value: 1.4729701996034248 and parameters: {'degree': 2}. Best is trial 0 with value: 1.4729701996034248.
[I 2024-02-05 00:16:24,093] Trial 5 finished with value: 1.4729701996034248 and parameters: {'degree': 3}. Best is trial 0 

In [12]:
best_model2 = SVC(degree= study2.best_params['degree'])
best_model2.fit(mordred_train, activityTrain)

SVC(degree=2)

In [13]:
y_pred = best_model2.predict(mordred_test)
accuracy2 = accuracy_score(y_true= activityTest, y_pred= y_pred)
print(accuracy2)

0.6812903225806451


In [14]:
tn, fp, fn, tp = confusion_matrix(y_pred, activityTest).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')


Accuracy = 0.6812903225806451
Sensitivity = 0.6835278858625162
Specificity = 0.25


In [15]:
model_type = "SVM_Mordred_Eye_Irritaion"
model_name = f"{model_type}{datetime.datetime.now().isoformat()}"
model_path = "../models/svm"
file = os.path.join(model_path, model_name)
pickle.dump(best_model2, open(file, 'wb'))

In [16]:
result_path = "../results/metrics.csv"
if not os.path.exists(result_path):
    print("Create metrics.csv file in the results folder")
    
def append_row(df, row):
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)
db_result = pd.read_csv(result_path)

if not db_result[db_result["model_name"] == model_type].size > 0:
    #db_result.loc[len(db_result)] == ["X", "yahkjahg", "kljgasdf", "kjadf"] 
    newRow = pd.Series({'model_name': model_type, 'ACC' : ACC , 'SEN' : SEN, 'SPE': SPE})
    db_result = append_row(db_result, newRow) 
else:
    db_result[db_result["model_name"] == model_type]["ACC"] = ACC
    db_result[db_result["model_name"] == model_type]["SEN"] = SEN
    db_result[db_result["model_name"] == model_type]["SPE"] = SPE
    
    
db_result.to_csv(result_path, index= False)
     